## Implementation of a Convolution Neural Network in Tensorflow
author : m.mahyar_ali ......date_created : April,2,2020

In [35]:
#import Libraries
import tensorflow as tf
import numpy as np

In [4]:
def create_placeholders(n_h,n_w,n_c,n_y):
    '''n_h : Height of filter matrix
       n_w : width of filter matrix
       n_c : Depth of the filter matrix
       n_y : Size of output layer'''
    #None is used for number of training examples,so that we can change it later
    X = tf.placeholder(tf.float32,shape=(None,n_h,n_w,n_c))
    Y = tf.placeholder(tf.float32,shape=(None,n_y))
    
    return X,Y

In [8]:
def initialize_parameters():
    '''
    We are using eight filters of 4*4 sie at CONV1 or layer 1
    W1 : [4, 4, 3, 8]
    
    We are using 16 filters of 2*2 size at layer 2 or CONV2
    W2 : [2, 2, 8, 16] 
    '''
    tf.set_random_seed(1)  #For Testing Purpose
    W1 = tf.get_variable('W1',[4, 4, 3, 8],initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable('W2',[2, 2, 8, 16],initializer=tf.contrib.layers.xavier_initializer(seed=0))
    
    parameters={'W1':W1,'W2':W2}
    
    return parameters

In detail, we will use the following parameters for all the steps:
 - Conv2D: stride 1, padding is "SAME"
 - ReLU
 - Max pool: Use an 8 by 8 filter size and an 8 by 8 stride, padding is "SAME"
 - Conv2D: stride 1, padding is "SAME"
 - ReLU
 - Max pool: Use a 4 by 4 filter size and a 4 by 4 stride, padding is "SAME"

In [14]:
def forward_propagation(X,parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    Z1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding="SAME")
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1,ksize=[1,8,8,1],strides=[1,8,8,1],padding="SAME")
    
    Z2 = tf.nn.conv2d(P1,W2,strides=[1,1,1,1],padding='SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2,ksize=[1,4,4,1],strides=[1,4,4,1],padding='SAME')
    
    F = tf.contrib.layers.flatten(P2)
    
    Z3 = tf.contrib.layers.fully_connected(F,6,activation_fn = None)
    #Cost Function will automatically compute the actiavtions
    return Z3

In [19]:
def compute_cost(Z3,Y):
    
    cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3,labels=Y))
    
    return cost 

In [33]:
def random_mini_batches(X_tarin,Y_train,minibatch_sizde,seed):
    np.random.seed(seed) ;
    m = X_train.shape[0] 
    
    #According to this case
    random_perm = list(np.random.perm(m))
    X_train = X_train[random_perm,:,:,3]
    Y_train = Y_train[random_perm,:]
    
    num_minibatches = int(m / minibatch_size)
    minibatches = []
    for batch in range(num_minibatches):
        minibatch_X = X_train[batch*minibatch_size:minibatch_size(batch+1),:,:,:]
        minibatch_Y = Y_train[batch*minibatch_size:minibatch_size(batch+1),:,:,:]
        minibatches.append((minibatch_X,minibatch_Y))
        
    # For thehttps://mpgntkofxqkwukrnvbolrk.coursera-apps.org/notebooks/week1/Untitled.ipynb?kernel_name=python3# last incomplete mini batch
    
    if m%minibatch_size != 0 :
        minibatch_X = X_train[minibatch_size*num_minibatches:,:,:,:]
        minibatch_Y = Y_train[minibatch_size*num_minibatches:,:,:,:]
        
        minibatch_Y = Y_train[batch*minibatch_size:minibatch_size(batch+1),:,:,:]
        minibatches.append((minibatch_X,minibatch_Y))

In [34]:
def model(X_train,Y_train,X_test,Y_test,learning_rate = 0.009,num_epoch=100,minibatch_size = 64,print_cost=True):
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    costs = []
    
    (m, n_h, n_w,n_c) = X_train.shape
    n_y = Y_train.shape[1]
    X ,Y = create_placeholders(n_h,n_w,n_c,n_y)
    
    parameters = initialize_parameters()
    
    Z3 = forward_propagation(X,parameters)
    
    cost = compute_cost(Z3,Y)
    
    optimizer= tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(num_epoch):
            minibatch_cost = 0
            seed = seed+1
            num_minibatches = int(m / minibatch_size)
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _,temp_cost = sess.run([optimizer,cost],feed_dict = {X:minibatch_X,Y:minibatch_Y})
                
                minibatch_cost += temp_cost / num_minibatches
                
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
            
    return parameters